# Existe relação diretamente o IBC-Br e o PIB?

# O que é o PIB?

O Produto Interno Bruto (PIB) representa o total de todos os bens e serviços finais gerados em um país, estado ou cidade, tipicamente calculados anualmente. Cada nação avalia seu PIB em sua própria moeda. Por exemplo, o PIB do Brasil alcançou R\$ 10,1 trilhões em 2022, e no terceiro trimestre de 2023, este valor atingiu R\$ 2.741,0 bilhões.

Para calcular o PIB, apenas os bens e serviços finais são contabilizados, evitando assim a contagem dupla. Por exemplo, se um país produzir trigo no valor de R\$ 100, farinha de trigo por R\$ 200 e pão por R\$ 300, o PIB será contado como R\$ 300. Isso acontece porque o valor do trigo e da farinha já está incluído no preço final do pão.

Importante destacar, o PIB mede esses bens e serviços finais a preços de consumidor, o que inclui os impostos embutidos nos produtos comercializados.

É um equívoco comum confundir o PIB com a riqueza total de um país. O PIB não reflete o 'estoque' de riqueza existente, mas sim um 'fluxo' de novos bens e serviços produzidos em um determinado período. Portanto, se em um ano um país não produzir nada, seu PIB será zero.

# O que é o IBC-Br?

O Índice de Atividade Econômica do Banco Central (IBC-Br) é uma métrica desenvolvida pelo Banco Central do Brasil (BACEN) para monitorar e antecipar tendências na economia brasileira. Ele serve como um indicador de desempenho econômico, refletindo a atividade em diversos setores da economia em um determinado período, geralmente mensal.

O IBC-Br é calculado com base em uma variedade de indicadores, incluindo os níveis de produção industrial, vendas no varejo e serviços, entre outros. Ao contrário do Produto Interno Bruto (PIB), que é calculado trimestralmente, o IBC-Br fornece uma visão mais frequente e atualizada da economia.

Este índice é particularmente útil para os formuladores de políticas econômicas e investidores, pois oferece uma visão preliminar da direção em que a economia está se movendo, antes mesmo da divulgação oficial do PIB. Isso permite reações e ajustes mais rápidos às tendências econômicas.

É importante destacar que, embora o IBC-Br seja um indicador abrangente, ele não mede todos os aspectos da economia. Por exemplo, ele não inclui dados sobre a economia informal. Além disso, assim como o PIB, o IBC-Br não deve ser confundido com um indicador de riqueza ou bem-estar, mas sim como um reflexo do nível de atividade econômica.

---

Após compreendermos os conceitos e os fatores determinantes que influenciam cada um dos indicadores econômicos, uma questão relevante emerge: existe uma forte correlação entre o IBC-Br, considerado uma prévia do PIB, e o próprio PIB? Para responder a esta pergunta, vamos usar o Python como ferramenta de coleta, tratamento, análise e modelagem dos dados para verificar esse argumento.

## Importando bibliotecas

In [8]:
# Pandas: Data manipulation and analysis library.
# https://pandas.pydata.org
try:
    import pandas as pd
except ModuleNotFoundError:
    print('Pandas is not installed, installing now...')
    %pip install pandas
finally:
    import pandas as pd

In [9]:
# Matplotlib: Data visualization library
# https://matplotlib.org
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    print('Matplotlib is not installed, installing now...')
    %pip install matplotlib
finally:
    import matplotlib.pyplot as plt

In [10]:
# Plotly: Interactive data visualization library
# https://plotly.com
try:
    import plotly.express as px
except ModuleNotFoundError:
    print('Plotly is not installed, installing now...')
    %pip install plotly
finally:
    import plotly.express as px

In [11]:
# Sidrapy: Brazilian IBGE data extraction library
# https://sidrapy.readthedocs.io/pt-br/latest
try:
    import sidrapy as sd
except ModuleNotFoundError:
    print('Sidrapy is not installed, installing now...')
    %pip install sidrapy
finally:
    import sidrapy as sd

In [12]:
# Configuring Matplotlib to display graphics inline.
%matplotlib inline

In [13]:
# Mudando o motor de plotagem padrão do Pandas para o Plotly.
pd.options.plotting.backend = 'plotly'

## Índice de Atividade Econômica do Banco Central (IBC-Br)

In [14]:
# Para obter a série histórica do IBC-Br vamos utilizar a API de Dados Abertos do Banco Central do Brasil. 
ibc_br = pd.read_json('https://api.bcb.gov.br/dados/serie/bcdata.sgs.24363/dados?formato=json')
# Ajustando o formato da data e definindo-a como índice do DataFrame.
ibc_br['data'] = pd.to_datetime(ibc_br['data'], dayfirst=True, format='%d/%m/%Y')
ibc_br.set_index('data', inplace=True)

In [15]:
# Visualizando a série histórica do IBC-Br.
fig = ibc_br.plot(
    title='Índice de Atividade Econômica do Banco Central (IBC-Br) - com ajuste sazonal')
# Configurando o layout do gráfico.
fig.update_layout(xaxis_title='',
                  yaxis_title='Índice',
                  showlegend=False,
                  template='plotly_white')

## Produto Interno Bruto (PIB) sem ajuste sazonal

In [16]:
def get_pib_data_from_ibge(table_code: str) -> pd.DataFrame:
    """Obtém e processa dados do Produto Interno Bruto (PIB) a partir de uma tabela do IBGE.

    Utiliza a biblioteca Sidrapy para acessar os dados de uma tabela específica do IBGE. 
    A função transforma e ajusta esses dados para uma forma mais útil, focando-se nas colunas 
    de data e valor do PIB.

    Parâmetros:
    table_code (str): Código da tabela do IBGE que contém os dados do PIB desejados.

    Processo:
    1. Os dados são obtidos usando o método `get_table` do Sidrapy, especificando o nível territorial, 
       código territorial do IBGE, período e classificação.
    2. As colunas são renomeadas para serem mais descritivas e apenas as colunas relevantes são mantidas.
    3. O tipo da coluna 'Valor' é ajustado para float.
    4. A coluna 'data' é ajustada para indicar o mês correspondente ao trimestre.
    5. A coluna 'data' é convertida para o tipo datetime e definida como índice do DataFrame.

    Retorna:
    pandas.DataFrame: Um DataFrame com duas colunas, 'data' e 'PIB', onde 'data' é o índice e 
                      representa as datas (ajustadas para o mês correspondente ao trimestre) e 
                      'PIB' contém os valores do Produto Interno Bruto.

    Exemplo de uso:
    >>> pib_data = get_pib_data_from_ibge('1621')
    """

    # Usando o Sidrapy para obter os dados do PIB.
    response_raw = sd.get_table(table_code=table_code,
                                territorial_level="1",
                                ibge_territorial_code="all",
                                period="all",
                                classification="11255/90707")

    # Renomeando as colunas e selecionando apenas as colunas de interesse.
    # Substituindo os nomes das colunas pelo conteúdo da primeira linha.
    response_raw = response_raw.rename(columns=response_raw.iloc[0]).iloc[1:]

    # Ajustando o tipo da coluna 'Valor' para float.
    response_raw['Valor'] = response_raw['Valor'].astype(float)

    # Selecionando apenas as colunas de interesse.
    response = response_raw.rename(columns={'Valor': 'PIB',
                                            'Trimestre (Código)': 'data'})[['data', 'PIB']]

    # Transformando a indicação de trimestre para o mês correspondente.
    response['data'] = response['data'].str[:-2] + response['data'].str[-2:].replace({"01": "03",
                                                                                      "02": "06",
                                                                                      "03": "09",
                                                                                      "04": "12"})

    # Ajustando o formato da data e definindo-a como índice do DataFrame.
    response['data'] = pd.to_datetime(response['data'], format='%Y%m')
    response.set_index('data', inplace=True)

    return response

In [17]:
# Usando o Sidrapy para obter a tabela 1620 do IBGE.
# Tabela: 1620 - Série encadeada do índice de volume trimestral (Base: média 1995 = 100)
pib = get_pib_data_from_ibge('1620')

# Visualizando a série histórica do PIB.
fig = pib.plot(title='Produto Interno Bruto (PIB) - sem ajuste sazonal')
fig.update_layout(xaxis_title='',
                  yaxis_title='Índice',
                  showlegend=False,
                  template='plotly_white')

## Produto Interno Bruto (PIB) com ajuste sazonal

In [18]:
# Usando o Sidrapy para obter a tabela 1621 do IBGE.
# Tabela: 1621 - Série encadeada do índice de volume trimestral com ajuste sazonal (Base: média 1995 = 100)
pib_sa = get_pib_data_from_ibge('1621')

# Visualizando a série histórica do PIB com ajuste sazonal.
fig = pib_sa.plot(title='Produto Interno Bruto (PIB) - com ajuste sazonal')
fig.update_layout(xaxis_title='',
                  yaxis_title='Índice',
                  showlegend=False,
                  template='plotly_white')

## Comparando o PIB e o PIB dessazonalizado

In [19]:
# Criando um DataFrame contemplando as duas séries históricas.
pib_df = pd.merge(pib, pib_sa, how='outer', left_index=True, right_index=True)

# Renomeando as colunas.
pib_df.columns = ['PIB', 'PIB_SA']

In [20]:
fig = pib_df.plot(title='Produto Interno Bruto (PIB) - com e sem ajuste sazonal')
fig.update_layout(xaxis_title='',
                  yaxis_title='Índice',
                  template='plotly_white')

# Como comparar o IBC-br e o PIB?

Para efetuar uma comparação adequada entre o IBC-Br e o PIB, é necessário realizar ajustes específicos nos dados de ambos os índices. Nosso foco será analisar as **variações marginais** de cada um deles. Isso significa que estamos interessados em entender como pequenas alterações em um período influenciam as mudanças observadas em ambos os indicadores. Este exercício nos permitirá avaliar como o IBC-Br, frequentemente visto como um preditor do PIB, reflete as flutuações econômicas e sua correspondência com as alterações registradas no PIB.

O caminho para realizar a comparação passar por dois passo específicos:
1. Calcular a média do trimestre
2. Calcular a variação marginal do trimestre

In [21]:
# Agrupando os dados por trimestre e calculando a média.
ibc_br_trimestral = ibc_br.resample('Q').mean()

# Calculando a variação percentual do IBC-Br em relação ao trimestre anterior.
ibc_br_variação_marginal = ibc_br_trimestral.diff()

In [22]:
# Calculando a variação percentual do PIB em relação ao trimestre anterior.
pib_variação_marginal = pib_df.diff()

In [23]:
# Padronizando as datas para o último dia mês.
pib_variação_marginal.index = pib_variação_marginal.index.to_period('M').to_timestamp('M')

In [24]:
# Criando um DataFrame contemplando todas as séries históricas.
df = pd.merge(ibc_br_variação_marginal, pib_variação_marginal.query(
    f"data >= '{ibc_br_variação_marginal.iloc[0].name.date().strftime('%Y-%m-%d')}'"),
    how='outer',
    left_index=True,
    right_index=True)

# Renomeando as colunas.
df.columns = ['IBC-Br', 'PIB', 'PIB_SA']

In [25]:
fig = df.plot(title='Variação percentual do IBC-Br e do PIB em relação ao trimestre anterior')
fig.update_layout(xaxis_title='',
                  yaxis_title='Variação percentual',
                  template='plotly_white')

Comparando a evolução temporal do IBC-Br e do PIB, nota-se que ambos os indicadores tendem a se mover em consonância, sugerindo uma sincronia nas suas flutuações. Essa observação nos leva a questionar se o IBC-Br pode ser considerado um preditor confiável do PIB. Para abordar essa questão de forma objetiva, é necessário empregar um modelo estatístico de regressão que possa quantificar a relação entre as séries temporais dos dois índices, levando em conta não apenas a correlação direta, mas também outros fatores que possam influenciar essa dinâmica.

## Disclaimer

O storytelling empregado neste estudo foi projetada com o intuito de tornar os dados apresentados e o método de elaboração compreensível para qualquer pessoa, independentemente de sua familiaridade com o assunto. Se você já possui conhecimento técnico em Python e/ou Ciência de Dados, sinta-se à vontade para ignorar quaisquer comentários que possam parecer redundantes ou enfadonhos.